In [1]:
%pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 7.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install mariadb

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# 1. 환경변수 로드
load_dotenv()

user = os.getenv('DB_USER', 'user1')
password = os.getenv('DB_PASSWORD', 'yttest1234')
host = os.getenv('DB_HOST', '121.128.172.79')
port = os.getenv('DB_PORT', '3306')
database = os.getenv('DB_NAME', 'yttest_db')

port = int(port)

# 2. CSV 파일 경로
csv_file = 'C:/Users/hp/Desktop/Bootcamp/PROJECT_OTT_AARRR/new_category_data_no_pos.csv'

# 3. CSV 파일 읽기
df = pd.read_csv(csv_file)
print("CSV 행 개수:", len(df))

# videoID 기준 중복 개수 확인
dup_counts = df.groupby('videoID').size().reset_index(name='count')
duplicated_ids = dup_counts[dup_counts['count'] > 1]
print("중복된 videoID:\n", duplicated_ids)

df.drop_duplicates(subset=['videoID'], keep='first', inplace=True)
print("중복 제거 후 행 개수:", len(df))

# 4. DB 테이블에 맞는 컬럼만 선택 및 순서 재정렬
#    DB 테이블 'video'의 컬럼:
#    [videoID, categoryID, channelID, title, viewCount, likeCount,
#     commentCount, uploadDate, duration, tags, thumbnailURL, keyword]
db_columns = [
    'videoID',
    'categoryID',
    'channelID',
    'title',
    'viewCount',
    'likeCount',
    'commentCount',
    'uploadDate',
    'duration',
    'tags',
    'thumbnailURL',
    'keyword'
]

# 불필요한 컬럼 제거
df = df[db_columns]

# 5. "None" 문자열을 결측치(NaN)로 변경 (숫자형 컬럼에서 문제가 될 수 있으므로)
df.replace("None", np.nan, inplace=True)

# 7. 숫자형 컬럼 변환 (오류가 발생하는 경우, 예를 들어 categoryID, viewCount, likeCount, commentCount, duration)
numeric_columns = ['categoryID', 'viewCount', 'likeCount', 'commentCount', 'duration']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 5. uploadDate 포맷 변환
# ISO 포맷(예: '2025-02-21T14:29:59Z')을 datetime으로 변환 후, 시간대 정보를 제거하고 'YYYY-MM-DD HH:MM:SS' 형식으로 변환
df['uploadDate'] = pd.to_datetime(df['uploadDate'], utc=True, errors='coerce')
df['uploadDate'] = df['uploadDate'].dt.tz_localize(None)
df['uploadDate'] = df['uploadDate'].dt.strftime('%Y-%m-%d %H:%M:%S')


# (변환 후, 각 숫자형 컬럼에 NaN이 포함되어 있는지 확인)
print(df[numeric_columns].head())

# 6. DB 연결을 위한 SQLAlchemy 엔진 생성
engine = create_engine(
    f"mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}",
    pool_recycle=3600
)

# 7. 외래 키 제약 조건 비활성화 후 데이터 삽입
with engine.begin() as conn:
    # 외래 키 제약 조건 비활성화
    conn.execute(text("SET FOREIGN_KEY_CHECKS=0;"))
    # 데이터 삽입 (chunksize 옵션 사용)
    df.to_sql('video', conn, if_exists='append', index=False, chunksize=1000)
    # 외래 키 제약 조건 재활성화
    conn.execute(text("SET FOREIGN_KEY_CHECKS=1;"))

print("CSV 파일의 데이터가 'video' 테이블에 성공적으로 적재되었습니다.")


CSV 행 개수: 101497
중복된 videoID:
             videoID  count
1919    0FHkzeKz3Vs      2
6023    2siR-j4ka6s      2
6253    30r_XifbGoA      2
6655    3IJEPlJ1Pn4      2
8537    4T1lM9EmZb4      2
...             ...    ...
97776   xfoKFm_caIw      2
98624   yEUcJsbLMoI      2
100627  zVOUI0mUiWs      2
100794  zanHKUv_ows      2
100903  zfTjontofLg      2

[101 rows x 2 columns]
중복 제거 후 행 개수: 101396
   categoryID  viewCount  likeCount  commentCount  duration
0           1    10876.0      738.0          46.0    3912.0
1           3      294.0        6.0           0.0     292.0
2           1      888.0      115.0           2.0      59.0
3           9    53186.0     2455.0         445.0     226.0
4           1      446.0        8.0           0.0      46.0
CSV 파일의 데이터가 'video' 테이블에 성공적으로 적재되었습니다.


In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# 1. 환경변수 로드
load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')

# 2. SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4"
)

# 3. 채널 데이터 불러오기 및 저장
channel_df = pd.read_csv("channels_by_keywords_최종.csv")

# DB에 저장
channel_df.to_sql(name='Channel', con=engine, if_exists='append', index=False)
print("Channel 테이블에 데이터 삽입 완료")

# 4. 영상 데이터 불러오기
video_df = pd.read_csv("videos_by_keywords_최종.csv", encoding='utf-8')

# segment 컬럼 제거 (있다면)
if 'segment' in video_df.columns:
    video_df = video_df.drop(columns=['segment'])

# uploadDate 포맷 변환
video_df['uploadDate'] = pd.to_datetime(video_df['uploadDate'], utc=True, errors='coerce')
video_df['uploadDate'] = video_df['uploadDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

# 컬럼명 정제
video_df.columns = ['videoID', 'channelID', 'title', 'viewCount', 'likeCount', 'commentCount',
                    'uploadDate', 'duration', 'tags', 'thumbnailURL', 'keyword', 'categoryID']

# 5. 채널 ID 유효한 것만 필터링
video_df = video_df.drop_duplicates(subset='videoID')
with engine.connect() as conn:
    valid_channels = pd.read_sql("SELECT channelID FROM Channel", conn)
valid_channel_ids = set(valid_channels['channelID'])

video_df = video_df[video_df['channelID'].isin(valid_channel_ids)]
print(f"삽입 가능한 영상 수: {len(video_df)}")

# 6. DB에 저장
video_df.to_sql(name='Video', con=engine, if_exists='append', index=False)
print("Video 테이블에 데이터 삽입 완료")


In [ ]:
# 1. channel 테이블에서 현재 존재하는 channelID 불러오기
valid_channel_ids = pd.read_sql("SELECT channelID FROM channel", con=engine)
valid_ids_set = set(valid_channel_ids['channelID'])

# 2. video_df에서 존재하지 않는 channelID만 필터링
invalid_channel_df = video_df[~video_df['channelID'].isin(valid_ids_set)]

# 3. 결과 확인
print(f"존재하지 않는 channelID 수: {invalid_channel_df['channelID'].nunique()}")
print(f"해당되는 영상 수: {len(invalid_channel_df)}")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from dotenv import load_dotenv
import os

# 환경 변수 불러오기
load_dotenv()
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')

# DB 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")

# CSV 파일 로드
pos_df = pd.read_csv('pos_tagged_data.csv')

# 컬럼 확인
print(pos_df.columns)

# videoID 기준 중복 제거
pos_df = pos_df.drop_duplicates(subset='videoID')

# DB 연결 후 업데이트
with engine.begin() as conn:
    for _, row in pos_df.iterrows():
        video_id = row['videoID']
        pos_tags = row['pos_tags']

        # SQLAlchemy text 쿼리 + 딕셔너리 파라미터
        conn.execute(
            text("""
                UPDATE video
                SET token_tagging = :token_tagging
                WHERE videoID = :videoID
            """),
            {"token_tagging": pos_tags, "videoID": video_id}
        )

print("video 테이블에 token_tagging 업데이트 완료.")